In [2]:
import numpy as np
import tensorflow as tf

http://kvfrans.com/variational-autoencoders-explained/

### implementation notes

* for last_dim, next_dim in zip(encode_dim, encode_dim[1:]): is very nice for iteration to get two terms at same time

  \[last_dim,next_dim  for last_dim, next_dim in zip(reversed(encode_dim), reversed(encode_dim[:-1]))\]
  
  

* Argue: should the encoder and decoder share bias term?
    - no, since they have different dimension
    - yes, since $f^{-1}$ should be exactly same with $f$

In [26]:
encode_dim = [784,400,200]
input_dim = encode_dim[0]
latent_dim = 20
num = 20
with tf.Graph().as_default():
    with tf.Session() as sess:
        input_x = tf.placeholder(dtype=tf.float32,shape=[num,input_dim])
        weight_list = []
        encode_bias_list = []
        decode_bias_list = []
        # init encoding weight,bias
        for last_dim, next_dim in zip(encode_dim, encode_dim[1:]):
            init_max_value = np.sqrt(6. / (last_dim + next_dim))
            weight_list.append(tf.Variable(tf.random_uniform([last_dim,next_dim],
                                                    np.negative(init_max_value),init_max_value)))
            encode_bias_list.append(tf.Variable(tf.random_uniform([next_dim],-0.1,0.1)))
        # init decoding bias
        for last_dim, next_dim in zip(reversed(encode_dim), reversed(encode_dim[:-1])):
            decode_bias_list.append(tf.Variable(tf.random_uniform([next_dim],-0.1,0.1)))
        init_max_value = np.sqrt(6. / (encode_dim[-1] + latent_dim))
        hidden_mean_weight = tf.Variable(tf.random_uniform([encode_dim[-1],latent_dim],
                                                    np.negative(init_max_value),init_max_value))
        hidden_mean_bias = tf.Variable(tf.random_uniform([latent_dim],
                                                    np.negative(init_max_value),init_max_value))
        hidden_variance_weight = tf.Variable(tf.random_uniform([encode_dim[-1],latent_dim],
                                                    np.negative(init_max_value),init_max_value))
        hidden_variance_bias = tf.Variable(tf.random_uniform([latent_dim],
                                                    np.negative(init_max_value),init_max_value))
        decode_weight = tf.Variable(tf.random_uniform([latent_dim,encode_dim[-1]],
                                                     np.negative(init_max_value),init_max_value))
        decode_bias = tf.Variable(tf.random_uniform([encode_dim[-1]],
                                                    np.negative(init_max_value),init_max_value))
        # Building Graph
        # encoder
        hidden_layer = input_x
        for weight,bias in zip(weight_list,encode_bias_list):
            hidden_layer = tf.nn.tanh(tf.matmul(hidden_layer,weight) + bias)
        # representation trick
        mean = tf.matmul(hidden_layer, hidden_mean_weight) + hidden_mean_bias
        variance = tf.matmul(hidden_layer, hidden_variance_weight) + hidden_variance_bias
        # sample from N(0,1)
        samples = tf.random_normal(tf.shape(variance),dtype=tf.float32,mean=0.0,stddev=1.0,name='samples')
        hidden_layer = mean + tf.exp(variance / 2) * samples
        hidden_layer = tf.nn.tanh(tf.matmul(hidden_layer,decode_weight) + decode_bias)
        # decoder
        for weight,bias in zip(reversed(weight_list),decode_bias_list):
            hidden_layer = tf.nn.sigmoid(tf.matmul(hidden_layer, tf.transpose(weight)) + bias)
        
        pass

In [15]:
encode_dim = [784,400,200]
for last_dim, next_dim in zip(encode_dim, encode_dim[1:]):
    print last_dim,next_dim
print "-----------------------"
for last_dim, next_dim in zip(reversed(encode_dim), reversed(encode_dim[:-1])):
    print last_dim,next_dim

784 400
400 200
-----------------------
200 400
400 784
